In [14]:
import os
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd


# Function to process selected months and CSV files
def process_selected_data(selected_months, selected_csvs, base_path):
    data = {}

    for month in selected_months:
        year, month_number = month.split('_')
        year_folder = f"{year}_V3"
        month_folder = f"{year}_{month_number}"
        month_path = os.path.join(base_path, year_folder, month_folder)

        print(f"Checking directory: {month_path}")
        if os.path.isdir(month_path):  # Ensure it is a directory
            print(f"Directory exists: {month_path}")
            month_data = {}
            for csv_type in selected_csvs:
                # Define the file path
                file_name_with_space = f"{csv_type}- {month_folder}.csv"
                file_name_without_space = f"{csv_type}-{month_folder}.csv"

                file_path_with_space = os.path.join(month_path, file_name_with_space)
                file_path_without_space = os.path.join(month_path, file_name_without_space)

                # Determine which file exists
                file_path = None
                if os.path.isfile(file_path_with_space):
                    file_path = file_path_with_space
                    print(f"Found file: {file_path}")
                elif os.path.isfile(file_path_without_space):
                    file_path = file_path_without_space
                    print(f"Found file: {file_path}")
                else:
                    print(f"File not found: {file_name_with_space} or {file_name_without_space} in {month_folder}")
                    continue  # Skip if neither file exists

                try:
                    # Read the CSV file
                    print(f"Attempting to load file: {file_path}")
                    data_frame = pd.read_csv(file_path, low_memory=False)
                    print(f"Successfully loaded {len(data_frame)} rows from {file_path}")

                    # Add metadata columns
                    data_frame['Source'] = csv_type
                    data_frame['Year'] = year
                    data_frame['Month'] = month_number

                    # Add to month-level data
                    month_data[csv_type] = data_frame

                except Exception as e:
                    print(f"Error loading file {file_path}: {e}")

            # Add the month data to the corresponding year
            if year not in data:
                data[year] = {}
            data[year][month] = month_data
        else:
            print(f"Directory does not exist: {month_path}")

    return data



# GUI for selecting months and CSV files
def selection_gui(base_path):
    def on_submit():
        selected_months = [month for month, var in month_checkboxes.items() if var.get()]
        selected_csvs = [csv for csv, var in csv_checkboxes.items() if var.get()]

        if not selected_months:
            messagebox.showwarning("No Selection", "Please select at least one month.")
            return
        if not selected_csvs:
            messagebox.showwarning("No Selection", "Please select at least one CSV file type.")
            return

        root.destroy()

        # Process data and create the hierarchical structure
        final_data = process_selected_data(selected_months, selected_csvs, base_path)

        # Display a preview of the processed structure in the terminal
        print("\nData Structure Overview:")
        for year, months in final_data.items():
            print(f"Year: {year}")
            for month, month_data in months.items():
                print(f"  Month: {month}")
                for csv_type, df in month_data.items():
                    print(f"    {csv_type}: {len(df)} rows")

    # Create the main GUI window
    root = tk.Tk()
    root.title("Select Months and CSV Files")
    root.geometry("1200x800")  # Set window size to maximize available screen space

    # Title label
    tk.Label(root, text="Select the months and CSV files you want to process:", font=("Arial", 16, "bold")).pack(pady=10)

    # Section for month selection
    month_frame = ttk.LabelFrame(root, text="Months", padding=(10, 10))
    month_frame.pack(fill="both", expand=True, padx=10, pady=5)

    month_checkboxes = {}
    row, col = 0, 0
    for year_folder in os.listdir(base_path):
        year_path = os.path.join(base_path, year_folder)
        if os.path.isdir(year_path):
            for month_folder in os.listdir(year_path):
                month_path = os.path.join(year_path, month_folder)
                if os.path.isdir(month_path):
                    var = tk.BooleanVar()
                    month_checkboxes[month_folder] = var
                    cb = tk.Checkbutton(month_frame, text=month_folder, variable=var, font=("Arial", 12))
                    cb.grid(row=row, column=col, sticky="w", padx=10, pady=5)
                    col += 1
                    if col >= 6:  # Change number of columns here to adjust layout
                        col = 0
                        row += 1

    # Section for CSV file selection
    csv_frame = ttk.LabelFrame(root, text="CSV Files", padding=(10, 10))
    csv_frame.pack(fill="both", expand=True, padx=10, pady=5)

    csv_checkboxes = {}
    csv_types = ["IVCurves", "LightSpectra", "SolarFieldData"]
    row = 0
    for csv_type in csv_types:
        var = tk.BooleanVar()
        csv_checkboxes[csv_type] = var
        cb = tk.Checkbutton(csv_frame, text=csv_type, variable=var, font=("Arial", 12))
        cb.grid(row=row, column=0, sticky="w", padx=10, pady=5)
        row += 1

    # Submit button
    tk.Button(root, text="Submit", command=on_submit, font=("Arial", 14), bg="green", fg="white").pack(pady=20)

    root.mainloop()




# Run the selection GUI
selection_gui(base_path)

# Call the function
df = load_data(base_path, selected_months, selected_csvs, debug=True)





NameError: name 'selected_months' is not defined

In [101]:
import os
import pandas as pd
from pathlib import Path

def load_and_sync_data(base_path, selected_months, selected_csvs, debug=False):
    base_path = Path(base_path)
    if not base_path.is_dir():
        raise FileNotFoundError(f"The base path does not exist: {base_path}")

    # Dictionary to store DataFrames from each source
    source_dataframes = {}

    for csv_type in selected_csvs:
        combined_source_data = []

        for month in selected_months:
            year, month_number = month.split('_')
            year_folder = f"{year}_V3"
            month_folder = f"{year}_{month_number}"
            month_path = base_path / year_folder / month_folder

            if not month_path.is_dir():
                if debug:
                    print(f"Skipping non-existent directory: {month_path}")
                continue

            # Determine the correct file name for the source
            file_name_with_space = f"{csv_type}- {month_folder}.csv"
            file_name_without_space = f"{csv_type}-{month_folder}.csv"

            file_path = None
            if csv_type == "LightSpectra":
                # Handle LightSpectra's unique naming format
                file_path_with_space = month_path / file_name_with_space
                file_path_without_space = month_path / file_name_without_space
                if file_path_with_space.exists():
                    file_path = file_path_with_space
                elif file_path_without_space.exists():
                    file_path = file_path_without_space
            else:
                file_path = month_path / file_name_without_space

            if file_path and file_path.exists():
                try:
                    data_frame = pd.read_csv(file_path, low_memory=False)

                    # -------------------
                    # Standardize date/time columns
                    # -------------------
                    data_frame["Year"] = data_frame.get("Date[year]", year)
                    data_frame["Month"] = data_frame.get("Date[month]", month_number)
                    data_frame["Day"] = data_frame.get("Date[day]")
                    data_frame["Weekday"] = data_frame.get("Date[weekday]")

                    if "Time[hh:mm:ss]" in data_frame.columns:
                        data_frame["Time"] = data_frame["Time[hh:mm:ss]"]
                        data_frame.drop(columns=["Time[hh:mm:ss]"], errors="ignore", inplace=True)

                    data_frame.drop(
                        columns=[col for col in data_frame.columns if col.startswith("Date[")],
                        errors="ignore",
                        inplace=True
                    )

                    data_frame["Source"] = csv_type  # (optional metadata)
                    combined_source_data.append(data_frame)

                except Exception as e:
                    if debug:
                        print(f"Error loading file {file_path}: {e}")
            else:
                if debug:
                    print(f"File not found: {file_path}")

        # Concatenate all data for the current source
        if combined_source_data:
            source_dataframes[csv_type] = pd.concat(combined_source_data, ignore_index=True)

    # -------------------
    # Merge (inner join) across all sources on "AbsTime[s]"
    # -------------------
    synced_data = None
    for source, df in source_dataframes.items():
        if synced_data is None:
            synced_data = df
        else:
            synced_data = pd.merge(
                synced_data,
                df,
                on='AbsTime[s]',
                how='inner',
                suffixes=('', f'_{source}')
            )

    # -------------------
    # Consolidate duplicate columns
    # -------------------
    if synced_data is not None:
        for col in ["Year", "Month", "Day", "Time", "Weekday"]:
            # Gather all columns that start with "Year", "Month", etc.
            cols_to_merge = [c for c in synced_data.columns if c.startswith(col)]
            if len(cols_to_merge) > 1:
                # Fill "master" col with first non-NaN going left->right
                synced_data[col] = synced_data[cols_to_merge].bfill(axis=1).iloc[:, 0]
                # Drop duplicates except for the original "col"
                cols_to_drop = [c for c in cols_to_merge if c != col]
                synced_data.drop(columns=cols_to_drop, inplace=True, errors="ignore")

        # Remove the "Source" columns if no longer needed
        synced_data.drop(
            columns=[col for col in synced_data.columns if "Source" in col],
            errors="ignore",
            inplace=True
        )

        # Remove exact duplicated columns (rare leftover edge case)
        synced_data = synced_data.loc[:, ~synced_data.columns.duplicated()]

        return synced_data

    # If no data was found at all
    return pd.DataFrame()


if __name__ == "__main__":
    # Base path to the dataset
    base_path = "../../OneDrive - HvA/Jaar_4/PV systems modeling and analysis/Data/data_folder"

    # Specify months and sources to include
    selected_months = get_selected_months(years=[2017, 2018], months=[1])
    selected_csvs = ["LightSpectra", "IVCurves"]

    # Load and synchronize data
    synced_df = load_and_sync_data(base_path, selected_months, selected_csvs, debug=True)

    # Display the synchronized DataFrame
    print("\nSynchronized DataFrame:")
    print(synced_df.head())

    # # Save the synchronized data for later use (optional)
    # synced_df.to_csv("synced_data_january_2017_2018.csv", index=False)
    # print("Synchronized data saved to synced_data_january_2017_2018.csv")



Synchronized DataFrame:
   AbsTime[s] Memo (Spectrometer) Sensor (Spectrometer)  \
0  3692217600           EKO WISER                MS-711   
1  3692217900           EKO WISER                MS-711   
2  3692218200           EKO WISER                MS-711   
3  3692218500           EKO WISER                MS-711   
4  3692218800           EKO WISER                MS-711   

   Exposure time[milliseconds] (Spectrometer)  \
0                                      5000.0   
1                                      5000.0   
2                                      5000.0   
3                                      5000.0   
4                                      5000.0   

   Sensor temperature[°C] (Spectrometer)  Power[V] (Spectrometer)  \
0                                   24.4                      5.9   
1                                   24.3                      5.9   
2                                   24.5                      5.9   
3                                   24.3         

In [102]:
synced_df

,AbsTime[s],Memo (Spectrometer),Sensor (Spectrometer),Exposure time[milliseconds] (Spectrometer),Sensor temperature[°C] (Spectrometer),Power[V] (Spectrometer),Wavelength list[nm] (Spectrometer),Irradiance list[W/m²/um] (Spectrometer),APE[eV] (Spectrometer),Year,...,I list[A]_4 (Solarpanel curves),average G [W/m²]_4 (Solarpanel curves),Module #_5 (Solarpanel curves),V list[V]_5 (Solarpanel curves),I list[A]_5 (Solarpanel curves),average G [W/m²]_5 (Solarpanel curves),Module #_6 (Solarpanel curves),V list[V]_6 (Solarpanel curves),I list[A]_6 (Solarpanel curves),average G [W/m²]_6 (Solarpanel curves)
0,3692217600,EKO WISER,MS-711,5000.0,24.4,5.9,287.77;288.21;288.65;289.08;289.52;289.95;290....,-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;0;0;-0;-0;-0;...,1.541894,2017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3692217900,EKO WISER,MS-711,5000.0,24.3,5.9,287.77;288.21;288.65;289.08;289.52;289.95;290....,0;0;-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;...,1.595728,2017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3692218200,EKO WISER,MS-711,5000.0,24.5,5.9,287.77;288.21;288.65;289.08;289.52;289.95;290....,-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;0;-0;0;-0;...,1.575863,2017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3692218500,EKO WISER,MS-711,5000.0,24.3,5.9,287.77;288.21;288.65;289.08;289.52;289.95;290....,0;0;0;0;0;0;0;0;0;0;0;-0;-0;0;-0;0;0;-0.01;-0....,1.625292,2017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3692218800,EKO WISER,MS-711,5000.0,24.5,5.9,287.77;288.21;288.65;289.08;289.52;289.95;290....,-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;-...,1.505024,2017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17851,3726430500,EKO WISER,MS-711,5000.0,24.4,5.9,287.77;288.21;288.65;289.08;289.52;289.95;290....,-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;0...,1.503984,2018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17852,3726430800,EKO WISER,MS-711,5000.0,24.5,5.9,287.77;288.21;288.65;289.08;289.52;289.95;290....,0;0;0;-0;0;-0;0;0;0;-0;-0;0;-0;0;-0;0;-0.01;-0...,1.567887,2018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17853,3726431100,EKO WISER,MS-711,5000.0,24.5,5.9,287.77;288.21;288.65;289.08;289.52;289.95;290....,-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;-...,1.555236,2018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17854,3726431400,EKO WISER,MS-711,5000.0,24.3,5.9,287.77;288.21;288.65;289.08;289.52;289.95;290....,-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;-0;0;-0;-0;-0...,1.614811,2018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
